In [8]:
import pandas as pd

unique_key = ["off_last_name","off_first_name","off_middle_initial","off_rank","off_star","off_race","off_year_of_birth","off_appointed"]
profs_df = pd.DataFrame(columns=unique_key)
intermediary_dfs = []

with pd.read_csv("../files/events/assignment_cleaned.csv", chunksize=10000) as reader:
  for chunk in reader:
    profs_df = pd.concat([profs_df, chunk.loc[:,unique_key]]).drop_duplicates(subset=unique_key)

profs_df.sort_values(by=["off_last_name", "off_first_name", "off_middle_initial"], inplace=True, ignore_index=True)
profs_df.to_csv("../files/events/officer_profs_test.csv")